In [ ]:
import json

with open('AskUbuntuCorpus.json') as json_file:
    data = json.load(json_file)
    
sentences = []
intents = []

classes = [
    'make_update',
    'none',
    'setup_printer',
    'shutdown_computer',
    'software_recommendation']

for sentence in data["sentences"]:
    sentences.append(sentence["text"].lower().strip())
    _class = sentence["intent"].lower().strip().replace(" ","_")
    intents.append(classes.index(_class))

print("Sentences:", len(sentences))
print("Intents:", intents)

In [2]:
import numpy as np
import tensorflow.compat.v2 as tf
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
from transformers import *

MODEL = "distilbert-base-uncased"

tokenizer = DistilBertTokenizer.from_pretrained(MODEL)
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL,
                                                              dropout=0.2,
                                                              num_labels=5)

In [3]:
tokenized_sequence = [
    tokenizer.encode(i, add_special_tokens=False) for i in sentences
]

In [4]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(
    tokenized_sequence,
    maxlen=16,
    dtype='int32',
    padding='pre',
    truncating='pre',
    value=0.0
)

In [5]:
y_train = np.asarray(intents, dtype="int")

In [6]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, decay=0.01)
opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt, "dynamic")
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")

model.compile(optimizer=opt, loss=loss, metrics=[metric])

model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  3845      
_________________________________________________________________
dropout_19 (Dropout)         multiple                  0         
Total params: 66,957,317
Trainable params: 66,957,317
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(x_train, y_train,
                    validation_data=(x_train, y_train,),
                    batch_size=8, epochs=10)

Train on 162 samples, validate on 162 samples
Epoch 1/10


/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


162/162 [==============================] - 11s 67ms/sample - loss: 1.4810 - accuracy: 0.4012 - val_loss: 1.2695 - val_accuracy: 0.5926
Epoch 2/10
162/162 [==============================] - 1s 5ms/sample - loss: 1.1307 - accuracy: 0.6358 - val_loss: 0.8558 - val_accuracy: 0.7654
Epoch 3/10
162/162 [==============================] - 1s 5ms/sample - loss: 0.7834 - accuracy: 0.8580 - val_loss: 0.5635 - val_accuracy: 0.9321
Epoch 4/10
162/162 [==============================] - 1s 5ms/sample - loss: 0.5375 - accuracy: 0.9321 - val_loss: 0.3763 - val_accuracy: 0.9444
Epoch 5/10
162/162 [==============================] - 1s 5ms/sample - loss: 0.3996 - accuracy: 0.9383 - val_loss: 0.2934 - val_accuracy: 0.9506
Epoch 6/10
162/162 [==============================] - 1s 5ms/sample - loss: 0.3157 - accuracy: 0.9506 - val_loss: 0.2379 - val_accuracy: 0.9506
Epoch 7/10
162/162 [==============================] - 1s 5ms/sample - loss: 0.2530 - accuracy: 0.9506 - val_loss: 0.1970 - val_accuracy: 0.9506
E

In [8]:
model.save_weights("../bert_intent.h5")